In [1]:
%cd ..

G:\My Drive\0-Research-Project\MedStar\MS_CODE\FieldNN


# Introduction


This is the for 

* module `fieldnn.basicnn.expander` module
* module `fieldnn.configfn.expanderfn` module

# Real Tensor

In [2]:
import os
from recfldgrn.datapoint import load_df_data_from_folder
from fieldnn.utils.layerfn import traverse, convert_relational_list_to_numpy

###################### take this as given
batch_PID_order = ['P1', 'P4', 'P5', 'P6']
######################

TaskTensor_folder = 'data/ProcData/TensorFolder/Task2YearXXX'
print(TaskTensor_folder)

l = sorted([i for i in os.listdir(TaskTensor_folder) if 'Grn' in i])
l

data/ProcData/TensorFolder/Task2YearXXX


['A1C@DT-DTDftGrn',
 'A1C@V-A1CNumeDftGrn',
 'Diag@DT-DTDftGrn',
 'Diag@Value-DiagDftGrn',
 'EC@BasicInfo-BasicDftGrn',
 'EC@DT_min-DTDftGrn',
 'P@age-AgeNumeDftGrn',
 'P@basicInfo-basicInfoDftGrn',
 'PN@DT-DTDftGrn',
 'PNSect@SectName-PNSctNmDftGrn',
 'PNSectSent@Sentence-Tk@TknzLLMGrn',
 'Smoking@DT-DTDftGrn',
 'Smoking@V-SmokingDftGrn']

In [3]:
recfldgrn_list = [
                  'A1C@V-A1CNumeDftGrn',
                  'Diag@DT-DTDftGrn',
                  'Diag@Value-DiagDftGrn',
                  'PNSectSent@Sentence-Tk@TknzLLMGrn'
                ]

In [4]:
batch_rfg = {}

for recfldgrn in recfldgrn_list:
    
    # (1) get tensor_folder
    tensor_folder = os.path.join(TaskTensor_folder, recfldgrn)
    print(tensor_folder)

    # (2) get df_Pat and full_recfldgrn
    df_Pat = load_df_data_from_folder(tensor_folder).set_index('PID')
    full_recfldgrn = df_Pat.columns[0]
    suffix = full_recfldgrn.split('_')[-1]
    assert recfldgrn in full_recfldgrn

    # (3) load batch: TODO: convert this to DataSet and DataLoader
    df_batch = df_Pat.loc[batch_PID_order]

    # (4) tensor batch as tensor_idx
    new_full_recfldgrn = 'B-' + full_recfldgrn
    values_list = df_batch[full_recfldgrn].to_list()
    suffix = full_recfldgrn.split('_')[-1]
    # print(suffix)
    # print(new_full_recfldgrn)
    D = convert_relational_list_to_numpy(values_list, new_full_recfldgrn, suffix)
    tensor_idx = D[new_full_recfldgrn]
    
    batch_rfg[new_full_recfldgrn] = tensor_idx
    
for k, v in batch_rfg.items(): print(k, v.shape)

data/ProcData/TensorFolder/Task2YearXXX\A1C@V-A1CNumeDftGrn
data/ProcData/TensorFolder/Task2YearXXX\Diag@DT-DTDftGrn
data/ProcData/TensorFolder/Task2YearXXX\Diag@Value-DiagDftGrn
data/ProcData/TensorFolder/Task2YearXXX\PNSectSent@Sentence-Tk@TknzLLMGrn
B-P-EC-A1C@V-A1CNumeDftGrn_wgt (4, 25, 1, 37)
B-P-EC-Diag@DT-DTDftGrn_idx (4, 25, 22, 7)
B-P-EC-Diag@Value-DiagDftGrn_idx (4, 25, 22, 3)
B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLMGrn_idx (4, 25, 1, 14, 121, 11)


## Analyze One RecFldGrn

**Understand Holder**


Introduction to Holder.

You can treat the holder as the position holder.

One holder is always with a info (tensor vectors).

We can have two types of holder:

(a) the grain_idx. **for example: **
    
```python
holder: [[123,   333,  123,  333], [ 56,  24,  23, 0],[ 24,  23, 0]]
info:   [[v123, v333, v123, v333], [v56, v24, v23, 0],[v24, v23, 0]]  
```

(b) the length of chidren for the focal element. **for example: after we process the above tensor to the sentence-level tensor. we will have this:**
       
```python
holder: [4,   3,  2]
info:   [s1, s2, s3]  # (here s1 is a sentence-level vector from [v123, v333, v123, v333]
```

**Relationships between leng_mask, leng, holder, and info**

1. Current Layer:

```python
leng_mask = holder == 0 # 
# for leng_mask: if with real value, then False, if with padding, then True
# in the real mask, we need to unsqueeze() the last dim. 
leng = (leng_mask == 0).sum(-1) # in leng_mask: 0 means a real position
``` 

2. Transfer

```python # 
holder = leng # (for next iteration)
```


3. Next Layer
```python
leng_mask = holder == 0
leng = (leng_mask == 0).sum(-1) # in leng_mask: 0 means a real position
```


In [5]:
import torch

full_recfldgrn = 'B-P-EC-Diag@Value-DiagDftGrn_idx'
info_idx = batch_rfg[full_recfldgrn]
print(info_idx.shape)
holder = torch.LongTensor(info_idx)

(4, 25, 22, 3)


In [6]:
print(full_recfldgrn, '<-- full_recfldgrn')

recfld = [i for i in full_recfldgrn.split('-') if '@' in i][0]
rec, fld = recfld.split('@')
grn_suffix = [i for i in full_recfldgrn.split('-') if 'Grn' in i][0]
grn, suffix = grn_suffix.split('_')
prefix_ids = [i for i in full_recfldgrn.split('-') if 'Grn' not in i and '@' not in i]
recfldgrn = rec + '@' + fld + '-' + grn

print(recfldgrn, '<---- recfldgrn')
print(prefix_ids, '<---- recfldgrn')
print(rec, '<---- rec')
print(fld, '<---- fld')
print(grn, '<---- grn')
print(suffix, '<---- suffix')

B-P-EC-Diag@Value-DiagDftGrn_idx <-- full_recfldgrn
Diag@Value-DiagDftGrn <---- recfldgrn
['B', 'P', 'EC'] <---- recfldgrn
Diag <---- rec
Value <---- fld
DiagDftGrn <---- grn
idx <---- suffix


# BasicNN Name

Expander try to convert GRN_idx (or GRN_wgt) to embedding vectors.

It is from idx to vector.

It is a type of order-increase. 


Here the core include: CateEmbedding, NumeEmbedding, and LLMEmbed. 



## CateEmbedding

### Module

In [7]:
import os
import torch
import numpy as np

class CateEmbeddingLayer(torch.nn.Module):

    def __init__(self, 
                 vocab_size, 
                 embedding_size, 
                 init = 'random', 
                 freeze = False):
        
        super(CateEmbeddingLayer, self).__init__()
        
        # create embedding
        if init == 'random':
            # c. initial from random initialization
            self.embedding = torch.nn.Embedding(vocab_size, embedding_size, padding_idx = 0)
        
        elif type(init) == np.ndarray:
            # a. load from pretrained array. Here init is an array.
            weight = torch.FloatTensor(init)
            assert weight.shape == (vocab_size, embedding_size)
            self.embedding = torch.nn.Embedding.from_pretrained(weight, freeze = freeze)
            
        elif os.path.isfile(init):
            # b. load from the pretrained array file.
            weight = torch.FloatTensor(np.load(init))
            assert tuple(weight.shape) == (vocab_size, embedding_size)
            self.embedding = torch.nn.Embedding.from_pretrained(weight, freeze = freeze)
        
        else:
            raise ValueError(f'In correct init method "{init}"')
        
    def forward(self, holder):
        # info is the grain
        info = self.embedding(holder)
        return info

### Config

In [8]:
### Main Config
### others will change with each recfldgrn

#######################
embed_size = 512
init = 'random'
#######################

In [9]:
def get_cateembed_para(embed_size, vocab_tokenizer, init = 'random'):
    embed_para =  {'embedding_size': embed_size,
                   'init': init, 
                   'vocab_size': len(vocab_tokenizer)}
    return embed_para

### Usage 

In [10]:
import pandas as pd

# (1) full recfldgrn information (we have discussed it already)
full_recfldgrn = 'B-P-EC-Diag@Value-DiagDftGrn_idx'

recfld = [i for i in full_recfldgrn.split('-') if '@' in i][0]
rec, fld = recfld.split('@')
grn_suffix = [i for i in full_recfldgrn.split('-') if 'Grn' in i][0]
grn, suffix = grn_suffix.split('_')
prefix_ids = [i for i in full_recfldgrn.split('-') if 'Grn' not in i and '@' not in i]
recfldgrn = rec + '@' + fld + '-' + grn


In [11]:
# (2) get vocab information, in the `recfldgrn` task, we have already created the FldGrnInfo
fldgrn_folder = 'data/ProcData/FldGrnInfo'
fullfldgrn_file = os.path.join(fldgrn_folder, rec + '.p')
df_FieldGrainInfo = pd.read_pickle(fullfldgrn_file)

# (3) Query the v2idx_tokenizer information from the df_FieldGrainInfo
# here v2idx could be a vocab dictionary for categorical data, or tokenizer for the texutal data.
vocab_tokenizer = df_FieldGrainInfo[df_FieldGrainInfo['recfield2grain'] == recfldgrn].iloc[0]['Vocab']['v2idx']

# (4) get the vocab_size for the v2idx_tokenizer. 
# ps. even tokenizer (from huggingface) can have use len().
vocab_size = len(vocab_tokenizer)
print(vocab_size)

801


**get configuration**


Here we want to get the embedding configuration.

In [12]:
embed_para = get_cateembed_para(embed_size, vocab_tokenizer, init)
print(embed_para)

{'embedding_size': 512, 'init': 'random', 'vocab_size': 801}


**init NN model**

In [13]:
if suffix == 'idx':
    NN = CateEmbeddingLayer(**embed_para)
    print(NN)
else:
    raise ValueError(f'suffix is not correct "{suffix}"')

CateEmbeddingLayer(
  (embedding): Embedding(801, 512, padding_idx=0)
)


**prepare input**

In [14]:
# (1) get the info_raw from batch_rfg
info_raw = batch_rfg[full_recfldgrn]
print(info_raw.shape, '<---- info_raw for full_recfldgrn' )

# (2) get the holder (input_idx) and holder_wgt (for nume embedding only)

# here you may wonder what is holder.
# You can treat the holder as the position holder.
# One holder is always with a info (tensor vectors).

# We can have two types of holder:
# (a) the grain_idx 
#     for example: 
#        holder: [[123,   333,  123,  333], [ 56,  24,  23, 0],[ 24,  23, 0]]
#        info:   [[v123, v333, v123, v333], [v56, v24, v23, 0],[v24, v23, 0]]  
#
# (b) the length of chidren for the focal element.
#     for example: after we process the above tensor to the sentence-level tensor.
#                  we will have this:
#        holder: [4,   3,  2]
#        info:   [s1, s2, s3]  (here s1 is a sentence-level vector from [v123, v333, v123, v333]
# 

# depending on the input type, we will create holder, and holder_wgt accordingly.
if suffix == 'idx':
    holder_wgt = 'Empty'
    holder = torch.LongTensor(info_raw)
elif suffix == 'wgt':
    holder_wgt = torch.FloatTensor(info_raw)
    # ATTENTION: here holder_wgt could contain zeros in some valid positions.
    holder = torch.ones_like(holder_wgt).cumsum(-1).masked_fill(holder_wgt == 0, 0).long()
else:
    raise ValueError(f'Invalid suffix "{suffix}"')
    
print(holder.shape, '<------ holder shape')
if type(holder_wgt) == str: 
    print(holder_wgt, '<------ holder wgt information')
else:
    print(holder_wgt.shape, '<------ holder wgt information')

(4, 25, 22, 3) <---- info_raw for full_recfldgrn
torch.Size([4, 25, 22, 3]) <------ holder shape
Empty <------ holder wgt information


**I-NN-O**

Here I-NN-O means `Input - (Neural Network) --> Output`. 

In [15]:
print(holder.shape, '<----- holder shape')
info = NN(holder)
print(info.shape, '<----- info shape')

if type(holder_wgt) != str:
    info = info * holder_wgt.unsqueeze(-1)
    print(info.shape, '<----- updated by holder_wgt')

torch.Size([4, 25, 22, 3]) <----- holder shape
torch.Size([4, 25, 22, 3, 512]) <----- info shape


## NumeEmbedding

### Module


Actually, `NumeEmbedding` is just the same as the `CateEmbedding`. 

However, in the future, we can defintely create the new customized embeddings for the numeric rec@fld grains. 

In [16]:
import os
import torch
import numpy as np

class NumeEmbeddingLayer(torch.nn.Module):

    def __init__(self, 
                 vocab_size, 
                 embedding_size, 
                 init = 'random', 
                 freeze = False):
        
        super(NumeEmbeddingLayer, self).__init__()
        
        # create embedding
        if init == 'random':
            # c. initial from random initialization
            self.embedding = torch.nn.Embedding(vocab_size, embedding_size, padding_idx = 0)
        
        elif type(init) == np.ndarray:
            # a. load from pretrained array. Here init is an array.
            weight = torch.FloatTensor(init)
            assert weight.shape == (vocab_size, embedding_size)
            self.embedding = torch.nn.Embedding.from_pretrained(weight, freeze = freeze)
            
        elif os.path.isfile(init):
            # b. load from the pretrained array file.
            weight = torch.FloatTensor(np.load(init))
            assert tuple(weight.shape) == (vocab_size, embedding_size)
            self.embedding = torch.nn.Embedding.from_pretrained(weight, freeze = freeze)
        
        else:
            raise ValueError(f'In correct init method "{init}"')
        
    def forward(self, holder):
        # info is the grain
        info = self.embedding(holder)
        return info

### Config

Actually, currently, the config function for the NumeEmbedding is exactly the same as the CateEmbedding. 

In [17]:
# same as CateEmbedding

def get_numeembed_para(embed_size, vocab_tokenizer, init = 'random'):
    embed_para =  {'embedding_size': embed_size,
                   'init': init, 
                   'vocab_size': len(vocab_tokenizer)}
    return embed_para

### Usage

**recfldgrn information**

In [18]:
# (1) full recfldgrn information
full_recfldgrn = 'B-P-EC-A1C@V-A1CNumeDftGrn_wgt'
recfld = [i for i in full_recfldgrn.split('-') if '@' in i][0]
rec, fld = recfld.split('@')
grn_suffix = [i for i in full_recfldgrn.split('-') if 'Grn' in i][0]
grn, suffix = grn_suffix.split('_')
prefix_ids = [i for i in full_recfldgrn.split('-') if 'Grn' not in i and '@' not in i]
recfldgrn = rec + '@' + fld + '-' + grn



# (2) get vocab information
fldgrn_folder = 'data/ProcData/FldGrnInfo'
fullfldgrn_file = os.path.join(fldgrn_folder, rec + '.p')
df_FieldGrainInfo = pd.read_pickle(fullfldgrn_file)
vocab_tokenizer = df_FieldGrainInfo[df_FieldGrainInfo['recfield2grain'] == recfldgrn].iloc[0]['Vocab']['v2idx']
# TODO: also adding padding to v2idx
vocab_size = len(vocab_tokenizer)
print(vocab_size)

37


**get configuration**

In [19]:
embed_para = get_numeembed_para(embed_size, vocab_tokenizer, init)
print(embed_para)

{'embedding_size': 512, 'init': 'random', 'vocab_size': 37}


**init NN model**

In [20]:
if suffix == 'idx':
    NN = CateEmbeddingLayer(**embed_para)
    print(NN)
elif suffix == 'wgt':
    NN = NumeEmbeddingLayer(**embed_para)
    print(NN)
else:
    raise ValueError(f'suffix is not correct "{suffix}"')

NumeEmbeddingLayer(
  (embedding): Embedding(37, 512, padding_idx=0)
)


**prepare input**

In [21]:
# (1) get the info_raw from batch_rfg
info_raw = batch_rfg[full_recfldgrn]
print(info_raw.shape)

# (2) get the holder (input_idx) and holder_wgt (for nume embedding only)
if suffix == 'idx':
    holder_wgt = 'Empty'
    holder = torch.LongTensor(info_raw)
elif suffix == 'wgt':
    holder_wgt = torch.FloatTensor(info_raw)
    # ATTENTION: here holder_wgt could contain zeros in some valid positions.
    holder = torch.ones_like(holder_wgt).cumsum(-1).masked_fill(holder_wgt == 0, 0).long()
else:
    raise ValueError(f'Invalid suffix "{suffix}"')
    
print(holder.shape, '<------ holder shape')
if type(holder_wgt) == str: 
    print(holder_wgt, '<------ holder wgt information')
else:
    print(holder_wgt.shape, '<------ holder wgt information')


(4, 25, 1, 37)
torch.Size([4, 25, 1, 37]) <------ holder shape
torch.Size([4, 25, 1, 37]) <------ holder wgt information


In [22]:
holder[0, 0, 0]

tensor([ 0,  0,  0,  0,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
        19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36,
         0])

In [23]:
 holder_wgt[0, 0, 0]

tensor([ 0.,  0.,  0.,  0., 11.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
         1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
         1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.])

**I-NN-O**

In [24]:
print(holder.shape, '<----- holder shape')
info = NN(holder)
print(info.shape, '<----- info shape')

if type(holder_wgt) != str:
    info_wgt = info * holder_wgt.unsqueeze(-1)
    print(info_wgt.shape, '<----- updated by holder_wgt')

torch.Size([4, 25, 1, 37]) <----- holder shape
torch.Size([4, 25, 1, 37, 512]) <----- info shape
torch.Size([4, 25, 1, 37, 512]) <----- updated by holder_wgt


In [25]:
holder[0, 0, 0]

tensor([ 0,  0,  0,  0,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
        19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36,
         0])

In [26]:
holder_wgt[0, 0, 0]

tensor([ 0.,  0.,  0.,  0., 11.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
         1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
         1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.])

In [27]:
info[0, 0, 0][:, 0]

tensor([ 0.0000,  0.0000,  0.0000,  0.0000, -0.0326, -0.5205, -0.7393,  0.4972,
         1.1058,  0.0822,  0.6718,  1.5901, -0.2112,  0.1481,  1.2041, -0.8224,
         0.4747,  0.5145, -0.1339, -2.1765, -0.6981,  1.5316, -0.3123, -0.9307,
         0.3096, -0.1674,  0.9595, -2.0493, -0.2045, -1.1049, -1.2157,  1.3847,
         1.0433,  0.8592,  1.7395, -0.7385,  0.0000],
       grad_fn=<SelectBackward0>)

In [28]:
info_wgt[0, 0, 0][:, 0]

tensor([ 0.0000,  0.0000,  0.0000,  0.0000, -0.3589, -0.5205, -0.7393,  0.4972,
         1.1058,  0.0822,  0.6718,  1.5901, -0.2112,  0.1481,  1.2041, -0.8224,
         0.4747,  0.5145, -0.1339, -2.1765, -0.6981,  1.5316, -0.3123, -0.9307,
         0.3096, -0.1674,  0.9595, -2.0493, -0.2045, -1.1049, -1.2157,  1.3847,
         1.0433,  0.8592,  1.7395, -0.7385,  0.0000],
       grad_fn=<SelectBackward0>)

## LLMEmbedding

### Module

In [29]:
import os
import torch
import numpy as np
# from ...utils.layerfn import orderSeq, restoreSeq
from fieldnn.utils.layerfn import orderSeq, restoreSeq
from transformers import AutoModel


class LLMEmbeddingLayer(torch.nn.Module):

    def __init__(self, 
                 tokenizer, 
                 embedding_size, 
                 init, 
                 freeze = False):
        
        super(LLMEmbeddingLayer, self).__init__()
        
        self.tokenizer = tokenizer
        assert init == tokenizer.name_or_path 
        
        self.LLM = AutoModel.from_pretrained(init)
        self.hidden_size = self.LLM.config.hidden_size
        self.embedding_size = embedding_size
        
        self.linear  = torch.nn.Linear(self.hidden_size, self.embedding_size)
        self.init_linear_weights()
            
    def init_linear_weights(self):
        initrange = 0.1
        self.linear.bias.data.zero_()
        self.linear.weight.data.uniform_(-initrange, initrange)
        
        
    def forward(self, holder):
        # 1. get leng_mask
        leng_mask = holder == 0
        
        # 2. get ordered holder
        ord_holder, ord_leng_mask, r_idx = self.reshape(holder, leng_mask)
        
        # 3. embedding ordered holder by LLM
        # expanding by the HuggingFace Language Model
        
        # 3.1 we might want to freeze LLM here
        output = self.LLM(ord_holder)
        # 3.2 adjust the hidden dimension
        ord_info_output = output['last_hidden_state']
        ord_info_output = self.linear(ord_info_output) # bias might not be zeros.
        ord_info_output = ord_info_output.masked_fill(ord_leng_mask.unsqueeze(-1), 0)
    
        # 4. restore orderded output to original shape
        info = self.restore(ord_info_output, leng_mask, r_idx)
        
        return info
    
    
    def reshape(self, holder, leng_mask):
        nbs = np.array(holder.shape[:-1]).prod()
        ngrn = holder.shape[-1]
        # print(nbs, ngrn, dim)
        
        tmp_holder = holder.contiguous().view(nbs, ngrn)
        # print(tmp_info.shape)

        tmp_leng_mask = leng_mask.contiguous().view(nbs, ngrn)
        # print(tmp_leng_mask.shape)

        tmp_leng = (tmp_leng_mask == 0).sum(-1)
        # print(tmp_leng.shape)
        
        ord_holder,    ord_leng, r_idx = orderSeq(tmp_holder, tmp_leng)
        ord_leng_mask, ord_leng, r_idx = orderSeq(tmp_leng_mask, tmp_leng)
        return ord_holder, ord_leng_mask, r_idx
    
    
    
    def restore(self, ord_info_output, leng_mask, r_idx):
        info_new = restoreSeq(ord_info_output, r_idx)
        output_size = info_new.shape[-1]
        info_output = info_new.view(*list(leng_mask.shape) + [output_size])
        return info_output
        

### Config

In [30]:
def get_llmembed_para(embed_size, tokenizer, init):
    embed_para =  {'embedding_size': embed_size,
                   'init': init, 
                   'tokenizer': tokenizer}
    return embed_para

### Usage

**recfldgrn information**

In [31]:
[i for i in batch_rfg]

['B-P-EC-A1C@V-A1CNumeDftGrn_wgt',
 'B-P-EC-Diag@DT-DTDftGrn_idx',
 'B-P-EC-Diag@Value-DiagDftGrn_idx',
 'B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLMGrn_idx']

In [32]:
# (1) full recfldgrn information
full_recfldgrn = 'B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLMGrn_idx'
recfld = [i for i in full_recfldgrn.split('-') if '@' in i][0]
rec, fld = recfld.split('@')
grn_suffix = [i for i in full_recfldgrn.split('-') if 'Grn' in i][0]
grn, suffix = grn_suffix.split('_')
prefix_ids = [i for i in full_recfldgrn.split('-') if 'Grn' not in i and '@' not in i]
recfldgrn = rec + '@' + fld + '-' + grn


# (2) get vocab information
fldgrn_folder = 'data/ProcData/FldGrnInfo'
fullfldgrn_file = os.path.join(fldgrn_folder, rec + '.p')
df_FieldGrainInfo = pd.read_pickle(fullfldgrn_file)

if 'LLM' in full_recfldgrn:
    vocab_tokenizer = df_FieldGrainInfo[df_FieldGrainInfo['recfield2grain'] == recfldgrn].iloc[0]['Vocab']['v2idx']
    # TODO: also adding padding to v2idx
    vocab_size = len(vocab_tokenizer)
    print(vocab_size)
else:
    vocab_tokenizer = df_FieldGrainInfo[df_FieldGrainInfo['recfield2grain'] == recfldgrn].iloc[0]['Vocab']['v2idx']
    # TODO: also adding padding to v2idx
    vocab_size = len(vocab_tokenizer)
    print(vocab_size)

30522


**get configuration**

In [33]:
init = vocab_tokenizer.name_or_path
embed_para = get_llmembed_para(embed_size, vocab_tokenizer, init)
embed_para

{'embedding_size': 512,
 'init': 'bert-base-uncased',
 'tokenizer': PreTrainedTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})}

**init NN model**

In [34]:
if suffix == 'idx' and 'LLM' not in recfldgrn:
    NN = CateEmbeddingLayer(**embed_para)
    print(NN)
    
elif suffix == 'idx' and 'LLM' in recfldgrn:
    NN = LLMEmbeddingLayer(**embed_para)
    
elif suffix == 'wgt':
    NN = NumeEmbeddingLayer(**embed_para)
    print(NN)
else:
    raise ValueError(f'suffix is not correct "{suffix}"')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


**prepare input**

In [35]:
# (1) get the info_raw from batch_rfg
info_raw = batch_rfg[full_recfldgrn]
print(info_raw.shape)

# (2) get the holder (input_idx) and holder_wgt (for nume embedding only)
if suffix == 'idx':
    holder_wgt = 'Empty'
    holder = torch.LongTensor(info_raw)
elif suffix == 'wgt':
    holder_wgt = torch.FloatTensor(info_raw)
    # ATTENTION: here holder_wgt could contain zeros in some valid positions.
    holder = torch.ones_like(holder_wgt).cumsum(-1).masked_fill(holder_wgt == 0, 0).long()
else:
    raise ValueError(f'Invalid suffix "{suffix}"')
    
print(holder.shape, '<------ holder shape')
if type(holder_wgt) == str: 
    print(holder_wgt, '<------ holder wgt information')
else:
    print(holder_wgt.shape, '<------ holder wgt information')


(4, 25, 1, 14, 121, 11)
torch.Size([4, 25, 1, 14, 121, 11]) <------ holder shape
Empty <------ holder wgt information


**I-NN-O**

In [36]:
print(holder.shape, '<----- holder shape')
info = NN(holder)
print(info.shape, '<----- info shape')

if type(holder_wgt) != str:
    info_wgt = info * holder_wgt.unsqueeze(-1)
    print(info_wgt.shape, '<----- updated by holder_wgt')

torch.Size([4, 25, 1, 14, 121, 11]) <----- holder shape
torch.Size([4, 25, 1, 14, 121, 11, 512]) <----- info shape


In [37]:
leng_mask = holder == 0
leng_mask.shape

torch.Size([4, 25, 1, 14, 121, 11])

In [38]:
info.shape

torch.Size([4, 25, 1, 14, 121, 11, 512])

#  *Basic NN Type

## Module

In [39]:
import torch

from fieldnn.basicnn.expander.cateembed import CateEmbeddingLayer
from fieldnn.basicnn.expander.numeembed import NumeEmbeddingLayer 
from fieldnn.basicnn.expander.llmembed import LLMEmbeddingLayer

# from .cateembed import CateEmbeddingLayer
# from .numeembed import NumeEmbeddingLayer 
# from .llmembed import LLMEmbeddingLayer

class Expander_Layer(torch.nn.Module):
    
    def __init__(self, input_names_nnlvl, output_name_nnlvl, expander_para):
        super(Expander_Layer, self).__init__()
        
        assert len(input_names_nnlvl) == 1
        self.input_names_nnlvl = input_names_nnlvl
        self.input_name_nnlvl = input_names_nnlvl[0]
        
        # output_name should be generated from the input_names
        self.output_name_nnlvl = output_name_nnlvl
        # self.output_name = output_name

        assert 'Grn' in self.input_name_nnlvl
        assert self.input_name_nnlvl.split('Grn')[0] == self.output_name_nnlvl
        
        # the input feature dim size and output feature dim size
        self.input_size = expander_para['input_size']
        self.output_size = expander_para['output_size']
        
        # Part 1: NN
        nn_name, nn_para = expander_para['nn_name'], expander_para['nn_para']
        
        if nn_name.lower() == 'cateembed':
            assert 'idx' in self.input_name_nnlvl and 'LLM' not in self.input_name_nnlvl
            self.Embed = CateEmbeddingLayer(**nn_para)
        elif nn_name.lower() == 'llmembed':
            assert 'idx' in self.input_name_nnlvl and 'LLM' in self.input_name_nnlvl
            self.Embed = LLMEmbeddingLayer(**nn_para)
        elif nn_name.lower() == 'numeembed':
            assert 'wgt' in self.input_name_nnlvl
            self.Embed = NumeEmbeddingLayer(**nn_para)
        else:
            raise ValueError(f'suffix is not correct "{self.input_name_nnlvl}"')

        self.embed_size = self.output_size
        
        # Part 2: PostProcess
        self.postprocess = torch.nn.ModuleDict()
        for method, config in expander_para['postprocess'].items():
            if method == 'dropout':
                self.postprocess[method] = torch.nn.Dropout(**config)
            elif method == 'activator':
                activator = config
                if activator.lower() == 'relu': 
                    self.postprocess[method] = torch.nn.ReLU()
                elif activator.lower() == 'tanh': 
                    self.postprocess[method] = torch.nn.Tanh()
                elif activator.lower() == 'gelu':
                    self.postprocess[method] = torch.nn.GELU()
            elif method == 'layernorm':
                self.postprocess[method] = torch.nn.LayerNorm(self.embed_size, **config)

    def forward(self, input_names_nnlvl, INPUTS_TO_INFODICT):
        '''
            info_dict
            input_names_nnlvl: full name of field, GRN is here
            holder: holder # i.e., info_idx
            holder_wgt: holder_wgt
        '''
        input_name_nnlvl = input_names_nnlvl[0]
        assert len(input_names_nnlvl) == 1
        assert input_name_nnlvl == self.input_name_nnlvl
        
        info_dict = INPUTS_TO_INFODICT[input_name_nnlvl]
        
        # 1. holder information
        holder = info_dict['holder']
        leng_mask = holder == 0
        embed = self.Embed(holder)
        
        # 2. holder_wgt information
        holder_wgt = info_dict['holder_wgt']
        if type(holder_wgt) != str:
            embed = embed * holder_wgt.unsqueeze(-1)
        
        # 3. post process.
        for nn, layer in self.postprocess.items():
            embed = layer(embed)
        
        return self.output_name_nnlvl, {'holder': holder, 'info': embed}
    

## Config

In [40]:
def get_expander_para(nn_name, nn_para,
                      embed_size, vocab_tokenizer, init, 
                      postprocess):
    
    expander_para = {}
    
    expander_para['nn_type'] = 'expander'
    expander_para['nn_name'] = nn_name

    # (1) get the parameters

    if nn_name.lower() == 'cateembed':
        para = get_cateembed_para(embed_size, vocab_tokenizer, init)
    elif nn_name.lower() == 'numeembed':
        para = get_numeembed_para(embed_size, vocab_tokenizer, init)
    elif nn_name.lower() == 'llmembed':
        para = get_llmembed_para(embed_size,  vocab_tokenizer, init)
    else:
        raise ValueError(f'The NN "{nn_name}" is not available yet')
        
    expander_para['nn_para'] = para
    
    
    #(2) Input size, output size
    expander_para['input_size'] = None
    expander_para['output_size'] = embed_size

    # (3) Post Process
    expander_para['postprocess'] = postprocess
    
    return expander_para